nous allons sur ce notebook pricer des options barrières, avec une formule fermée et une simulation de monte carlo on rappelle la relation d'arbitrage:
$Knockin(K,T,H) + Knockout(K,T,H)= Vanilla(K,T)$

$
\text{Payoff}_{UI, \text{call}} = \max(0, S(T) - K) \cdot \mathbb{1} \left( \max_{t \in [0,T]} S(t) \geq H \right)
$


In [25]:
import numpy as np
from scipy.stats import norm

def UIcallPrice(K, T, H, sigma, r, q, S):
  """
    Calcule le prix d'une option barrière up and in call selon la formule fermée.
    Paramètres :
    S : float spot
    K : float strike
    H: float barrière
    T : float maturité
    r : float Taux d'intérêt sans risque.
    q: float taux dividendes
    sigma : float Volatilité  sous-jacent.

    Retourne :
    float Prix de l'option.
    """
  #calcul des paramètres pour la formule fermée

  l=(r-q+(sigma**2)/2)/(sigma**2)

  y= np.log((H**2)/(K*S))/(sigma*np.sqrt(T))  + l*sigma*np.sqrt(T)

  x1= np.log(S/H)/(sigma*np.sqrt(T))    + l*sigma*np.sqrt(T)

  y1= np.log(H/S)/(sigma*np.sqrt(T))   + l*sigma*np.sqrt(T)
#calcul du prix

  prix = S*norm.cdf(x1)*np.exp(-q*T) + K*np.exp(-r*T)*norm.cdf(x1-sigma*np.sqrt(T))
  prix= prix - S*np.exp(-q*T)*(H/S)**(2*l)*(norm.cdf(-y1)-norm.cdf(-y))
  prix= prix + K*np.exp(-r*T)*(H/S)**(2*l-2)*(norm.cdf(-y+sigma*np.sqrt(T))-norm.cdf(-y1+sigma*np.sqrt(T)))

  return prix

# Exemple
S = 95  # spot
K = 100   # strike
H = 130 #barrière
r = 0.05  # Taux d'intérêt sans risque
q= 0.02 # taux de dividende
T = 1.0   # maturité
sigma = 0.2  # Volatilité


prix_callUI = UIcallPrice( K, T, H, sigma, r, q, S)

print(f"Prix de l'option call up and in : {prix_callUI:.2f}")





Prix de l'option call up and in : 2.31


In [26]:
def black_scholes(S, K, T, r, q, sigma, option_type="call"):
    """
    Calcule le prix d'une option européenne selon le modèle de Black-Scholes.
    Paramètres :
    S : float spot
    K : float strike
    T : float maturité
    r : float Taux d'intérêt sans risque.
    sigma : float Volatilité  sous-jacent.
    option_type : str type d'option : "call" ou "put"

    Retourne :
    float Prix de l'option européenne.
    """
    # Calcul des paramètres d1 et d2 pour black-scholes
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":

        prix = S*np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        # Prix de l'option de vente
        prix = K * np.exp(-r * T) * norm.cdf(-d2) - S*np.exp(-q*T) * norm.cdf(-d1)

    return prix

passons maitenant au calcul de l'up and out call avec les mêmes paramètres en s'apuyant sur la formule fermée pour l'up and in call
$
\text{Payoff}_{U0, \text{call}} = \max(0, S(T) - K) \cdot \mathbb{1} \left( \max_{t \in [0,T]} S(t) < H \right)
$

In [27]:
def UOcall(K,T,H,sigma,r,q,S):
  prix = black_scholes(S,K,T,r,q,sigma,option_type="call") - UIcallPrice(K, T, H, sigma, r, q, S)
  return prix

# Exemple
S = 95  # spot
K = 100   # strike
H = 130 #barrière
r = 0.05  # Taux d'intérêt sans risque
q= 0.02 # taux de dividende
T = 1.0   # maturité
sigma = 0.2  # Volatilité


prix_callUO = UOcall( K, T, H, sigma, r, q, S)


print(f"Prix de l'option call up and out : {prix_callUO:.2f}")


Prix de l'option call up and out : 4.23


La méthode de Monte Carlo repose sur la simulation d'un grand nombre de trajectoires du prix du sous-jacent afin d'estimer la valeur de l'option.

On génère plusieurs chemins du prix du sous-jacent en utilisant un Mouvement Brownien Géométrique (GBM).
Pour chaque trajectoire, on calcule la valeur finale de l'option en appliquant la formule de payoff.
On fait la moyenne des payoffs obtenus sur toutes les simulations.
On actualise cette moyenne au taux sans risque r
pour obtenir la valeur présente de loption :
$exp(-rt)* E[payoff]$
elle peut être longue à converger et nécessite un grand nombre de simulations pour obtenir une bonne précision mais possède bien plus de souplesse dans la réalisation que la formule ferméee qui reste pour un cas précis

In [31]:
import numpy as np
from scipy.stats import norm

def simuler_chemins_GBM(S0, r, q, sigma, T, pas, n_sims, graine=None):
    """
    Simule n_sims trajectoires du Mouvement Brownien Géométrique (GBM)
    avec un nombre donné de points de discrétisation.

    Paramètres :
    S0 : float, prix initial du sous-jacent
    r : float, taux d'intérêt sans risque
    q : float, taux de dividende en continu
    sigma : float, volatilité du sous-jacent
    T : float, maturité en années
    pas : int, nombre de sous-périodes de simulation
    n_sims : int, nombre de simulations Monte Carlo
    graine : int ou None, graine aléatoire pour reproductibilité

    Retourne :
    np.array de taille (n_sims, pas+1) contenant les chemins simulés.
    """
    if graine is not None:
        np.random.seed(graine)

    dt = T / pas  # Taille d'un pas de temps
    S = np.zeros((n_sims, pas + 1))  # Matrice des trajectoires
    S[:, 0] = S0  # Initialisation avec le prix spot

    for t in range(1, pas + 1):
        Z = np.random.normal(0, 1, n_sims)  # Bruit gaussien standard
        S[:, t] = S[:, t - 1] * np.exp((r - q - 0.5 * sigma**2) * dt
                                       + sigma * np.sqrt(dt) * Z)
    return S

def call_up_and_in_mc(S0, K, H, r, q, sigma, T, pas, n_sims, graine=None):
    """
    Calcule le prix d'une option call up-and-in par simulation Monte Carlo.

    Paramètres :
    S0 : float, prix initial du sous-jacent
    K : float, prix d'exercice
    H : float, barrière supérieure
    r : float, taux d'intérêt sans risque
    q : float, taux de dividende en continu
    sigma : float, volatilité du sous-jacent
    T : float, maturité en années
    pas : int, nombre de sous-périodes pour la simulation
    n_sims : int, nombre de simulations Monte Carlo
    graine : int ou None, graine aléatoire pour reproductibilité

    Retourne :
    float, estimation du prix de l'option call up-and-in.
    """
    # Simulation des trajectoires du sous-jacent
    chemins_S = simuler_chemins_GBM(S0, r, q, sigma, T, pas, n_sims, graine)

    # Valeur du sous-jacent à maturité (dernière colonne)
    S_T = chemins_S[:, -1]

    # Indicateur : la barrière a été atteinte (1 si max(S) >= H, 0 sinon)
    barriere_atteinte = (np.max(chemins_S, axis=1) >= H)

    # Payoff = max(S(T) - K, 0) si barrière atteinte, sinon 0
    payoffs = np.where(barriere_atteinte, np.maximum(S_T - K, 0), 0)

    # Actualisation au taux sans risque
    return np.exp(-r * T) * np.mean(payoffs)

def call_up_and_out_mc(S0, K, H, r, q, sigma, T, pas, n_sims, graine=None):
    """
    Calcule le prix d'une option call up-and-out par simulation Monte Carlo.

    Paramètres :
    S0 : float, prix initial du sous-jacent
    K : float, prix d'exercice
    H : float, barrière supérieure
    r : float, taux d'intérêt sans risque
    q : float, taux de dividende en continu
    sigma : float, volatilité du sous-jacent
    T : float, maturité en années
    pas : int, nombre de sous-périodes pour la simulation
    n_sims : int, nombre de simulations Monte Carlo
    graine : int ou None, graine aléatoire pour reproductibilité

    Retourne :
    float, estimation du prix de l'option call up-and-out.
    """
    # Simulation des trajectoires du sous-jacent
    chemins_S = simuler_chemins_GBM(S0, r, q, sigma, T, pas, n_sims, graine)

    # Valeur du sous-jacent à maturité
    S_T = chemins_S[:, -1]

    # Indicateur : la barrière n'a pas été atteinte (1 si max(S) < H, 0 sinon)
    barriere_non_atteinte = (np.max(chemins_S, axis=1) < H)

    # Payoff = max(S(T) - K, 0) si barrière non atteinte, sinon 0
    payoffs = np.where(barriere_non_atteinte, np.maximum(S_T - K, 0), 0)

    # Actualisation
    return np.exp(-r * T) * np.mean(payoffs)

if __name__ == "__main__":
    # Paramètres du modèle
    S0 = 95      # Prix spot initial
    K = 100      # Prix d'exercice
    H = 130      # Barrière supérieure
    r = 0.05     # Taux d'intérêt sans risque
    q = 0.02     # Taux de dividende en continu
    sigma = 0.2  # Volatilité
    T = 1.0      # Maturité en années
    pas = 252*24    # Nombre de pas (ex : 252 jours boursiers 24 h de fonctionnement)
    n_sims = 100000  # Nombre de simulations Monte Carlo

    prix_call_UI = call_up_and_in_mc(S0, K, H, r, q, sigma, T, pas, n_sims, graine=42)
    prix_call_UO = call_up_and_out_mc(S0, K, H, r, q, sigma, T, pas, n_sims, graine=42)

    print(f"Prix Monte Carlo du call up-and-in  : {prix_call_UI:.4f}")
    print(f"Prix Monte Carlo du call up-and-out : {prix_call_UO:.4f}")


Prix Monte Carlo du call up-and-in  : 3.7484
Prix Monte Carlo du call up-and-out : 2.7815


on observe de bien meilleur résultats avec la méthode de Monte-Carlo qu'avec la méthode en sollution fermée, cela peut dépendre de conditions non respecté pour la bonne utilisation de la solution fermé.